# Task 2: Data Profiling, Cleaning & EDA
**Objective:** Profile, clean, and explore the solar dataset for Benin so it’s ready for comparison and region-ranking tasks.

This notebook includes:
- Summary statistics and missing-value report
- Outlier detection and cleaning
- Time series analysis
- Correlation and scatter plots
- Wind and temperature analysis
- Bubble charts

## Importing the dependencies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

## Loading the Data

In [ ]:
# Set both plotting and display settings
import pandas as pd

# Load dataset
df = pd.read_csv(r"D:\Python\Week_01\data\data\benin-malanville.csv")

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 10)

In [ ]:
# Show basic info
print(f"Dataset shape: {df.shape}")
df.info()

In [ ]:
#  Display the first 5 rows
print("\nFirst 5 rows:")
display(df.head())

In [ ]:
# Display the last 10 rows
print("\nLast 10 rows:")
display(df.tail(10))

In [ ]:
# Display 10 random sample rows
print("\nRandom sample of 10 rows:")
display(df.sample(10, random_state=42))

In [ ]:
# Summary statistics for numeric columns
print("\nSummary statistics for numeric columns:")
display(df.describe())

In [ ]:
# Check for missing values
print("\nMissing values per column:")
print(df.isna().sum())

# Percentage of missing values per column
null_percent = df.isna().mean() * 100
print("\nPercentage of missing values:")
print((null_percent).round(2))

# Filter columns with more than 5% nulls
cols_with_nulls = null_percent[null_percent > 5].index.tolist()
print("\nColumns with >5% nulls:", cols_with_nulls)

# Exact duplicate rows
dup_count = df.duplicated().sum()
print("Duplicate rows:", dup_count)

# Cardinality (uniqueness) for categoricals
cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
cardinality = {c: df[c].nunique() for c in cat_cols}
print("Cardinality (categoricals):", cardinality)



## Outlier Detection and Cleaning

In [ ]:
# Univariate Analysis Numeric columns only
numeric_cols = df.select_dtypes(include=np.number).columns
print(df[numeric_cols].describe().T)

### Summary of Missing Values & Outliers in columns GHI, DNI, and DHI, ModA, ModB and WS, WSgust

In [ ]:
def check_solar_wind_outliers(df, time_col='Timestamp'):
    """
    Checks for missing values, negative/invalid values, and IQR outliers in key solar and wind columns.
    """
    
    df[time_col] = pd.to_datetime(df[time_col])
    
    # Columns and physical bounds based on the statistical summary
    columns_bounds = {
        'GHI': (0, 1400),
        'DNI': (0, 1000),
        'DHI': (0, 800),
        'ModA': (0, None),
        'ModB': (0, None),
        'WS': (0, 30),
        'WSgust': (0, 50)
    }
    
    summary = {}
    
    for col, (min_val, max_val) in columns_bounds.items():
        # Missing values
        missing_count = df[col].isna().sum()
        missing_percent = df[col].isna().mean() * 100
        
        # Negative or above max
        below_min = df[df[col] < min_val].shape[0] if min_val is not None else 0
        above_max = df[df[col] > max_val].shape[0] if max_val is not None else 0
        
        # IQR outliers
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        iqr_outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)].shape[0]
        
        # Summary
        summary[col] = {
            'missing_count': missing_count,
            'missing_percent': missing_percent,
            'below_min': below_min,
            'above_max': above_max,
            'iqr_outliers': iqr_outliers
        }
    
    return summary
summary_report = check_solar_wind_outliers(df)
for col, stats in summary_report.items():
    print(f"\nColumn: {col}")
    for k, v in stats.items():
        print(f"  {k}: {v}")

In [ ]:
# plot time series with highlighted outliers
columns_to_plot = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
# Spikes, negative values, or sudden jumps become visible.
for col in columns_to_plot:
    plt.figure(figsize=(12,4))
    plt.plot(df['Timestamp'], df[col], label=col, alpha=0.6)
    plt.title(f"{col} Time Series")
    plt.xlabel("Time")
    plt.ylabel(col)
    plt.show()